In [1]:
from mmdet.datasets import CocoDataset
import json
import os
import random
import numpy as np

In [2]:
dname = "aerial-maritime"
# dname = "pklot"
# dname = "d6-dice"

ann_file = f'/mnt/ssd2/sc_datasets_det/{dname}/annotations/instances_train.json'

with open(ann_file, "r") as fp:
    anno = json.load(fp)

In [3]:
annots = anno["annotations"]

N = len(annots)
M = max(int(N * 0.05), 1)
print(f"N={N}, M={M}")

random.shuffle(annots)

no_set = annots[:M]
ok_set = annots[M:]

N=1237, M=61


In [4]:
images = {ann["id"]: ann for ann in anno["images"]}
dxdys = [np.array([1, 0]), np.array([0, 1]), np.array([1, 1])] + [-np.array([1, 0]), -np.array([0, 1]), -np.array([1, 1])]
cat_ids = np.unique([ann["category_id"] for ann in annots])

def _move_bbox(ann, delta = 0.4):
    img_w, img_h = images[ann["image_id"]]["width"], images[ann["image_id"]]["height"]
        
    x, y, w, h = ann["bbox"]
    # x1, y1, x2, y2 = x, y, x + w, y + h
    wh = np.array([w, h])

    dxdy = random.choice(dxdys)
    dxdy = 0.2 * wh * dxdy
    
    x1y1 = np.array([x, y]) + dxdy
    x2y2 = x1y1 + wh

    x1, y1 = x1y1
    x2, y2 = x2y2

    x1 = np.clip(x1, 0, img_w)
    x2 = np.clip(x2, 0, img_w)
    y1 = np.clip(y1, 0, img_h)
    y2 = np.clip(y2, 0, img_h)

    ann["bbox"] = [x1, y1, x2 - x1, y2 - y1]
    ann["noise"] = 1

def _change_label(ann):
    id = ann["category_id"]
    new_id = random.choice(cat_ids)
    while id == new_id:
        new_id = random.choice(cat_ids)

    ann["category_id"] = id
    ann["noise"] = 2

for idx, ann in enumerate(no_set):
    if idx < M // 2:
        _move_bbox(ann)
    else:
        _change_label(ann)

for ann in ok_set:
    ann["noise"] = 0

anno["annotations"] = no_set + ok_set


In [5]:
dir_path, fname = os.path.split(ann_file)
name, ext = os.path.splitext(fname) 

new_fpath = os.path.join(dir_path, name + "_noise" + ext)

with open(new_fpath, "w") as fp:
    json.dump(anno, fp)
    print(new_fpath)

/mnt/ssd2/sc_datasets_det/aerial-maritime/annotations/instances_train_noise.json


In [2]:
from mmdet.datasets.builder import PIPELINES
from mmcv.parallel import DataContainer as DC
from mmdet.datasets.pipelines import to_tensor
from mmdet.datasets import CocoDataset
import json
import numpy as np

img_size = (992, 736)
img_norm_cfg = dict(mean=[0, 0, 0], std=[255, 255, 255], to_rgb=True)

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='MinIoURandomCrop',
        min_ious=(0.1, 0.3, 0.5, 0.7, 0.9),
        min_crop_size=0.3),
    dict(
        type='Resize',
        img_scale=[(992, 736), (896, 736), (1088, 736), (992, 672), (992, 800)],
        multiscale_mode='value',
        keep_ratio=False),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='DefaultFormatBundle'),
    dict(type="LabelNoiseBundle"),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels', 'noise_labels', 'anno_ids'])
]


@PIPELINES.register_module()
class LabelNoiseBundle(object):
    def __call__(self, results):
        ann_info = results["ann_info"]
        for key in ["noise_labels", "anno_ids"]:
            if key not in ann_info:
                continue
            results[key] = DC(to_tensor(ann_info[key]))
        return results
    
class LabelNoiseCocoDataset(CocoDataset):
    def __init__(self, min_size=None, *args, **kwargs):
        super().__init__(min_size, *args, **kwargs)

    def _parse_ann_info(self, img_info, ann_info):
        """Parse bbox and mask annotation.

        Args:
            ann_info (list[dict]): Annotation info of an image.
            with_mask (bool): Whether to parse mask annotations.

        Returns:
            dict: A dict containing the following keys: bboxes, bboxes_ignore,\
                labels, masks, seg_map. "masks" are raw annotations and not \
                decoded into binary masks.
        """
        gt_bboxes = []
        gt_labels = []
        gt_bboxes_ignore = []
        gt_masks_ann = []
        noise_labels = []
        anno_ids = []
        for i, ann in enumerate(ann_info):
            if ann.get('ignore', False):
                continue
            x1, y1, w, h = ann['bbox']
            inter_w = max(0, min(x1 + w, img_info['width']) - max(x1, 0))
            inter_h = max(0, min(y1 + h, img_info['height']) - max(y1, 0))
            if inter_w * inter_h == 0:
                continue
            if ann['area'] <= 0 or w < 1 or h < 1:
                continue
            if self.min_size is not None:
                if w < self.min_size or h < self.min_size:
                    continue
            if ann['category_id'] not in self.cat_ids:
                continue
            bbox = [x1, y1, x1 + w, y1 + h]
            if ann.get('iscrowd', False):
                gt_bboxes_ignore.append(bbox)
            else:
                gt_bboxes.append(bbox)
                gt_labels.append(self.cat2label[ann['category_id']])
                gt_masks_ann.append(ann.get('segmentation', None))
                noise_labels.append(ann["noise"])
                anno_ids.append(ann["id"])

        if gt_bboxes:
            gt_bboxes = np.array(gt_bboxes, dtype=np.float32)
            gt_labels = np.array(gt_labels, dtype=np.int64)
        else:
            gt_bboxes = np.zeros((0, 4), dtype=np.float32)
            gt_labels = np.array([], dtype=np.int64)

        if gt_bboxes_ignore:
            gt_bboxes_ignore = np.array(gt_bboxes_ignore, dtype=np.float32)
        else:
            gt_bboxes_ignore = np.zeros((0, 4), dtype=np.float32)

        seg_map = img_info['filename'].replace('jpg', 'png')

        ann = dict(
            bboxes=gt_bboxes,
            labels=gt_labels,
            bboxes_ignore=gt_bboxes_ignore,
            masks=gt_masks_ann,
            seg_map=seg_map,
            noise_labels=noise_labels,
            anno_ids=anno_ids
        )

        return ann

dname = "aerial-maritime"
ann_file = f'/mnt/ssd2/sc_datasets_det/{dname}/annotations/instances_train_noise.json'
img_prefix = f'/mnt/ssd2/sc_datasets_det/{dname}/images/train'

with open(ann_file, "r") as fp:
    anno = json.load(fp)

classes = [ele["name"] for ele in anno["categories"]]

dataset = LabelNoiseCocoDataset(
    ann_file=ann_file,
    pipeline=train_pipeline,
    classes=classes,
    data_root="",
    img_prefix=img_prefix,
    seg_prefix=None,
    proposal_file=None,
    test_mode=False,
    filter_empty_gt=True)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
